# Flipkart Web Scraping Using BeautifulSoup

* Flipkart is a E-Store where lot's of products available. Which makes it ideal site to scrap data from it. 
* This model of web scrapper, can scrap any type of product data for the given fields..
* This model can scrap not only one page but all the pages that flipkart can offer for a product.

#### Required libraries

We going to need BeautifulSoup, requests, html5lib and csv

In [ ]:
# Importing Libraries
from bs4 import BeautifulSoup
import requests
import html5lib
import time
import csv
# Do !pip install library_name if any library is missing

We need to provide URL of the page we need to scrap data from and the csv file name 

In [ ]:
# The url of the page
# We taking url of a page contains laptop as product
url = 'https://www.flipkart.com/search?q=laptop&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off'
# Name of the csv file the data going to stored in..
# It is best to name the file similar to the product 
file_name = 'flipkart_laptop_data.csv'

It is good to have functions made for each task. It makes it easy modify functions later on.

We going to start by making a function to get the page data 

In [ ]:
# Function to request and fetch the whole page
def getdata(url): # url is the url of the page we need to scrap data from
    r = requests.get(url) # this line of code will send a request to server to get data
    soup = BeautifulSoup(r.content, 'html5lib') # this line of code will store data in a variable 'soup'
    return soup 

As mentioned before, this programm is going to scrap data from all pages that flipkart going to provide as result

That is why we need a function that going to check for next page url and if there is any, it will return that url

The way to do that is, find the 'nav' having yFHi8N class in it. This class contains many a tags in it which a tags has urls' of different pages. The a tag which has _1LKTO3 class in it, either has url of previous page(if there is any previous page) of the next page(if there is any previous page). Now we need url of the next page only. Hence we need to run a for loop to go through all a tags with _1LKTO3 class and get the url if a tag has the url of next page.

In [ ]:
# Function to check and get next page url
def getnextpage(soup):
    try:
        page = soup.find('nav', {'class': 'yFHi8N'})
        tag = page.findAll('a', attrs = {'class': '_1LKTO3'}) # getting all a tag having _1LKTO3 class in it
        for i in tag:
            if i.text == 'Next': # Looking for 'Next' to get the next page url
                url = 'https://www.flipkart.com' + str(i['href']) #  this code add the href found in a tag and concat with 'https://www.flipkart.com' to get actual url
                return(url)
            else:
                continue
    except:
        return # there will no next page url at some point, in which case a tag will return Null value, which can crash the code. That's where try_except comes in

While trying to fetch different type of products data, it has been noticed that different products is being served in different way which means the html codes are different for different layouts. WHich makes it hard to scrap data of a random product type with a single type of code.

Now as the layouts are different so the div that contains the product, has different class. Now we can loop through all the class that are present and check for which class we get data out of a div. Then use that class to fetch data from div for a perticular product.

[Point to be noted : The classes those are considered as the all possible classes are found by manually checking different layouts. It is still possible that there are other classes. In which case the new class has to be added as an element of 'all_class' inside 'check_page_format(soup)' function. Also please let me know trough commemt if there is a new class]

In [ ]:
# function to check which class is valid for the given page
def check_page_format(soup):
    all_class = ['_2kHMtA','_4ddWXP','_1xHGtK _373qXS'] # All possible class that can be in a product div
    for class_type in all_class:
        if soup.find('div', attrs = {'class':class_type}) is not None:
            needed_class = class_type
        else:
            continue
    return needed_class

The function below does the job of visiting every single product page and gather the required data. 

First this function looks for url of every single product page. Then get the content of the product pages. This programm fetch only one product data at a time. 

For this project we getting Price, Name, Rating, Number of rating givers and Number of review givers.

First, we get the price as string data. Then remove the '₹'(rupee) sign and the ',' comma from it and convert it into int type. 

Second, we get Name of the product as string type.

Third, we get rating as float type.

Fourth, we get the number of review givers and number of rating givers as joint value. Then, seperate it and store into different variable. Then finally convert it into int type.

Fifth, we get the link of the image of the product and store it as string type.

Sixth, we get the product page link and store it as string type.

Then store all data in a dictonary.

In [ ]:
# function to fetch data of products
def fetch_data(needed_class, soup):
    single_page_all_product_data = [] # A empty list to store all data about all products
    for needed_class in soup.findAll('div', attrs = {'class':needed_class}):
        single_product_url = needed_class.a['href']
        single_product_data = {} # A dictonary to store all data of a single product
        sub_url = 'https://www.flipkart.com' + str(single_product_url) # url of the page of each product
        sub_r = requests.get(sub_url)
        time.sleep(0.5) # a sleep command to avoid spamming 
        sub_soup = BeautifulSoup(sub_r.content, 'html5lib') 
        # Getting price of the product
        price = sub_soup.find('div', {'class':'_30jeq3 _16Jk6d'})
        if price is not None:
            single_product_data['price'] = int(price.text.replace('₹','').replace(',','')) # A process to convert price data_type from string to int

        else:
            single_product_data['price'] = None
        # Getting Name of the product
        product_name = sub_soup.find('span', {'class':'B_NuCI'})
        if product_name is not None:
            single_product_data['product_name'] = product_name.text
        else:
            single_product_data['product_name'] = None
        # Getting rating of the product
        rating = sub_soup.find('div', {'class':'_3_L3jD'})
        if rating is not None:
            single_product_data['rating'] = float(rating.find('div', {'class':'_3LWZlK'}).text) # A process to convert rating data_type from string to float
        else:
            single_product_data['rating'] = 0.0 # There is no rating means the rating is zero
        # Getting No of rating and review gives
        no_of_rating_and_review_givers = sub_soup.find('span', {'class':'_2_R_DZ'})
        if no_of_rating_and_review_givers is not None:
            no_of_rating_and_review_givers = no_of_rating_and_review_givers.text
        else:
            no_of_rating_and_review_givers = '1&2' # A random value which going to get removed in next phase
        no_of_rating_and_review_givers = no_of_rating_and_review_givers.replace(',','').split('&')
        for i in no_of_rating_and_review_givers:
            if 'Ratings' in i:
                single_product_data['no_of_rating_givers'] = int(i.split(' ')[0]) # Saving only No of rating givers
            elif 'Reviews' in i:
                single_product_data['no_of_review_givers'] = int(i.split(' ')[0]) # Saving only No of review givers
            else :
                single_product_data['no_of_rating_givers'] = 0 # There is no rating givers means no of rating givers is zero
                single_product_data['no_of_review_givers'] = 0 # There is no review givers means no of rating givers is zero
        # Getting link of product image
        product_image = sub_soup.find('div', {'class':'_3kidJX'})
        if product_image is not None:
            single_product_data['image_link'] = product_image.img['src']
        else:
            single_product_data['image_link'] = None
        # Getting link of the product page
        single_product_data['product_page_link'] = sub_url
        single_page_all_product_data.append(single_product_data) # Store all product data in a dictonary
    return single_page_all_product_data

This function below makes a .csv file which will store all the data has been gathered.

In [ ]:
# function to store all data in a csv file
def convert_to_csv(all_product_data, file_name):
    with open(file_name, 'w', newline = '', encoding='UTF8') as f: # open file in write mode
        w = csv.DictWriter(f,['price', 'product_name', 'rating', 'no_of_rating_givers', 'no_of_review_givers', 'image_link', 'product_page_link'])
        w.writeheader() # Add header
        for product_data in all_product_data:
            w.writerow(product_data) # Add products data 

The fetch_data() function only returns data of products that is in one result page. There can be multiple result page as mentioned before. That is why, a empty list has been created which going to store all the data of products from all the pages.

As we don't know how many pages there going to be, using a infinite while loop is perfect for this senario. This loop will only break if there is no url found for next page. For example, if there is 3 page of result and no 4th page, when the getdata() function will scrp data of 3rd page products, the getnextpage() function will find the link of 4th page allthough there will be nothing in 4th page. Hence, the loop will run again. But this time getnextpagedata will not find any url there as 4th page is empty and will return 'Null' value as url. Due to invalid url the while loop will break and all gathered data will be stored into a .csv file. The .csv file location will be the same as this file.


In [ ]:
all_product_data = [] # A empty list to store data of all products from all result page
while True:
    data = getdata(url) # Fetch data from given URL
    url = getnextpage(data) # Get Next page URL
    
    if url: # Check if url has any value or not
        print(url)
        class_name = check_page_format(data) # Get the right format of the page
        page_data = fetch_data(class_name, data) # Get all product data from one page
        all_product_data.extend(page_data) # Merge all page data
        continue
    else:
        break
print(all_product_data) # Print all collected data
convert_to_csv(all_product_data, file_name) # Store data into a csv file

### Resources

* YouTube videos on web-scraping
* Github 
* Publicly available documents
* Python Community
* Web scraping community

### New fields are to be added soon